# Datenanalyse


---

Datensatz: [Synthea Breast Cancer Dataset](https://github.com/Fuenfgeld/DMA2023TeamA/tree/main/Daten/Quelldaten)

Primär- und Fremdschlüsseldefinitionen: [Synthea GitHub Repository](https://github.com/synthetichealth/synthea/wiki/CSV-File-Data-Dictionary)

Projektgruppe GitHub Repository: [DMA2023TeamA](https://github.com/Fuenfgeld/DMA2023TeamA)

Source-DB: [GoogleDrive Ablage](https://drive.google.com/drive/folders/1k5cfjGXjNHmwQkydzjTdVHoBvCniBU_W), erstellt mit [Setup_and_fill_Database.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/Setup_and_fill_Database.ipynb)

Data Warehouse-Datenbank: [GoogleDrive Ablage](https://drive.google.com/file/d/1l-HcqCezubHnR737DkbiRzdHanP7_g_D), erstellt mit [ETL_process.ipynb](https://github.com/Fuenfgeld/DMA2023TeamA/blob/main/Code/ETL_Process.ipynb)


*Version*: 0.2

Version Date: 07/02/2023

Changes: 

* Erstellung eines Analysedatensatzes aus den aus der DWH-Datenbank extrahierten Daten (Gesamtkosten für Aufenthalte / Prozeduren / Medikationen pro Patient)
* Explorative Datenanalyse (EDA)
* Ergänzung Versionsnummern für neue Pakete



# Vorbereitung des Notebooks

In [ ]:
# Vorsichtshalber: Löschen aller Variablen
%reset -f

# Laden der benötigten Libraries
from google.colab import drive
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
# will make plot outputs appear and stored within the notebook.
%matplotlib inline

In [ ]:
# Google Drive mounten, force_remount auf True setzen, damit ein Remount erzwungen wird
drive.mount('/content/gdrive/', force_remount=True)

# Datenbankordner auf dem Shareddrive checken, es müssen source_breast_cancer.db und DWH_breast_cancer.db vorhanden sein
!ls "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank"

# Patiententyp festlegen
patient_type = "breast_cancer"

# Pfad zur DWH-Datenbank setzen
DB_DWH_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Datenbank/DWH_breast_cancer.db"

# Pfad zum Analyse-Datensatz setzen
DATA_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Daten/Analyse/"

# Pfad zum Ergebnisordner setzen
RESULTS_PATH = "/content/gdrive/Shareddrives/DMA_Datenprojekt_TeamA/Ergebnisse/"

# Check
print("\n" + DB_DWH_PATH)
print("\n" + DATA_PATH)
print("\n" + RESULTS_PATH)

# Versionen der verwendeten Pakete abfragen 

Die Versionen der verwendeten Python-Installation und der Python-Pakete abfragen. 

In [ ]:
# Python-Version
print("Python-Version:")
!python --version

# Pandas-Version
print("\n" + "Pandas-Version:")
print("Pandas " + pd.__version__)

# numpy-Version
print("\n" + "numpy-Version:")
print("numpy " + np.version.version)

# sqlite-Version
print("\n" + "sqlite3-Version:")
print("sqlite3 " + sq.sqlite_version)

# Seaborn-Version
print("\n" + "Seaborn-Version:")
print("Seaborn " + sns.__version__)

# Matplotlib-Version
print("\n" + "Matplotlib-Version:")
print("Matplotlib " + matplotlib.__version__)


# Extraktion der Daten aus der DWH-Datenbank


In [ ]:
# Datenbankverbindung zum DWH aufbauen
dwh_conn = sq.connect(DB_DWH_PATH) 
if dwh_conn is not None:
  dwh_cursor = dwh_conn.cursor()
else:
  print("Verbindung fehlgeschlagen. Bitte überprüfen!")

# Alle Tabellennamen aus der Datenbank ziehen
dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tablelist = dwh_cursor.fetchall()
tablelist

In [ ]:
# SQL Abfrage zur Ausleitung der Daten zu den Behandlungskosten
extract_costs = """SELECT enc.enc_id,
                enc.patient,
                pat.gender    AS PAT_GENDER,
                pat.race      AS PAT_RACE,
                enc.enc_payer,
                pay.NAME      AS ENC_PAYER_NAME,
                enc.enc_base_cost,
                enc.enc_total_claim_cost,
                enc.enc_payer_coverage,
                enc.pro_code,
                enc.pro_base_cost,
                enc.med_code,
                enc.med_base_cost,
                enc.med_dispenses,
                enc.med_total_cost,
                enc.med_payer_coverage,
                enc.med_payer,
                enc.con_code,
                sct.term      AS CON_TERM
                FROM  f_encounter_costs AS enc
                      JOIN d_patients AS pat
                        ON enc.patient = pat.id
                      JOIN d_payers AS pay
                        ON enc.enc_payer = pay.id
                      LEFT JOIN d_snomedct AS sct
                        ON enc.con_code = sct.code
                ORDER  BY patient, enc.enc_id;"""
data_costs = pd.read_sql(extract_costs, dwh_conn)
data_costs.head()

In [ ]:
# SQL Abfrage zur Ausleitung der Diagnosedaten
extract_conditions = """SELECT enc.patient,
                               enc.con_start,
                               enc.con_code,
                               sct.term
                        FROM   f_encounter_costs AS enc
                               JOIN d_snomedct AS sct
                                 ON enc.con_code = sct.code
                        ORDER  BY enc.patient,
                                  enc.con_code;"""
# data_conditions = pd.read_sql(extract_conditions, dwh_conn)
# data_conditions.head()

In [ ]:
# SQL Abfrage zur Ausleitung der Diagnoseart (none = keine, breast_cancer = Brustkrebs, other = andere Diagnose)
extract_diagnoses = """SELECT pat.id AS patient,
                              CASE
                                WHEN pat.id IN (SELECT DISTINCT( enc.patient )
                                                FROM   f_encounter_costs AS enc
                                                WHERE  enc.con_code = "254837009") THEN 'breast_cancer'
                                WHEN pat.id IN (SELECT DISTINCT( enc.patient )
                                                FROM   f_encounter_costs AS enc
                                                WHERE  NOT enc.con_code = "254837009"
                                                        AND enc.con_code IS NOT NULL) THEN 'other'
                                ELSE 'none'
                              END    AS diagnosis
                        FROM   d_patients AS pat;"""
data_diagnoses = pd.read_sql(extract_diagnoses, dwh_conn)
data_diagnoses.head()

In [ ]:
# Commit und Close
dwh_conn.commit()
dwh_conn.close()

# Erstellung einer Analysetabelle

---



## Explorative Datenanalyse der aus der DWH-Datenbank erstellten Tabellen

In [ ]:
# Überblick über Tabelle data_costs
data_costs.info()

In [ ]:
# Anzahl einmaliger Werte pro Spalte in Tabelle data_costs
data_costs.nunique(axis=0)

In [ ]:
# Analyse Duplikate in Tabelle data_costs (je nach Kostentyp)
print("Anzahl doppelte Aufenthalte:", data_costs.duplicated(subset=["PATIENT", "ENC_ID", "ENC_BASE_COST", "ENC_PAYER_COVERAGE"]).sum())
print("Anzahl doppelte Prozeduren:", data_costs.duplicated(subset=["PATIENT", "ENC_ID", "PRO_CODE", "PRO_BASE_COST"]).sum())
print("Anzahl doppelte Medikationen:", data_costs.duplicated(subset=["PATIENT", "ENC_ID", "MED_CODE", "MED_BASE_COST", "MED_DISPENSES", "MED_TOTAL_COST", "MED_PAYER_COVERAGE"]).sum())

In [ ]:
# Überblick über Tabelle data_diagnoses
data_diagnoses.info()

In [ ]:
# Anzahl einmaliger Werte pro Spalte in Tabelle data_diagnoses
data_diagnoses.nunique(axis=0)

In [ ]:
# Analyse Duplikate in Tabelle data_diagnoses
print("Anzahl doppelte Patienten:", data_costs.duplicated().sum())

## Erstellung Tabelle mit Gesamtkosten für Aufenthalte pro Patient

In [ ]:
# Erstellung Subset mit Kostendaten zu Aufenthalten
data_encounters = data_costs[["PATIENT", "ENC_ID", "ENC_BASE_COST", "ENC_PAYER_COVERAGE"]]
data_encounters.shape

In [ ]:
# Entfernen von Duplikaten (gleicher Patient, gleicher Aufenthalt, gleiche Kosten)
data_encounters_nodups = data_encounters.drop_duplicates()
data_encounters_nodups.shape
print(len(data_encounters) - len(data_encounters_nodups), "Duplikate entfernt.")

In [ ]:
# Gruppierung nach Patienten, Aufsummierung der Kosten
data_encounters_grouped = data_encounters_nodups.groupby(by="PATIENT")
data_encounters_final = data_encounters_grouped[["ENC_BASE_COST", "ENC_PAYER_COVERAGE"]].sum()
data_encounters_final.rename(columns={"ENC_BASE_COST": "OVERALL_ENC_BASE_COST", 
                                        "ENC_PAYER_COVERAGE": "OVERALL_ENC_PAYER_COVERAGE"}, inplace=True)
data_encounters_final["NUMBER_ENCOUNTERS"] = data_encounters_grouped.size()
data_encounters_final.shape

In [ ]:
data_encounters_final

## Erstellung Tabelle mit Gesamtkosten für Prozeduren pro Patient

In [ ]:
# Erstellung Subset mit Kostendaten zu Prozeduren
data_procedures = data_costs[["PATIENT", "ENC_ID", "PRO_CODE", "PRO_BASE_COST"]]
data_procedures.shape

In [ ]:
# Entfernen von Duplikaten (gleicher Patient, gleicher Aufenthalt, gleiche Prozedur, gleiche Kosten)
data_procedures_nodups = data_procedures.drop_duplicates()
data_procedures_nodups.shape
print(len(data_procedures) - len(data_procedures_nodups), "Duplikate entfernt.")

In [ ]:
# Gruppierung nach Patienten, Aufsummierung der Kosten
data_procedures_grouped = data_procedures_nodups.groupby(by="PATIENT")
data_procedures_final = data_procedures_grouped[["PRO_BASE_COST"]].sum(min_count=1)
data_procedures_final.rename(columns={"PRO_BASE_COST": "OVERALL_PRO_BASE_COST"}, inplace=True)
data_procedures_final["NUMBER_PROCEDURES"] = data_procedures_grouped[["PRO_BASE_COST"]].count()
data_procedures_final.shape

In [ ]:
data_procedures_final

## Erstellung Tabelle mit Gesamtkosten für Medikation pro Patient

In [ ]:
# Erstellung Subset mit Kostendaten zu Medikationen
data_medications = data_costs[["PATIENT", "ENC_ID", "MED_CODE", "MED_BASE_COST", "MED_DISPENSES", "MED_TOTAL_COST", "MED_PAYER_COVERAGE"]]
data_medications.shape

In [ ]:
# Entfernen von Duplikaten (gleicher Patient, gleicher Aufenthalt, gleiche Medikation, gleiche Kosten)
data_medications_nodups = data_medications.drop_duplicates()
data_medications_nodups.shape
print(len(data_medications) - len(data_medications_nodups), "Duplikate entfernt.")

In [ ]:
# Gruppierung nach Patienten, Aufsummierung der Kosten
data_medications_grouped = data_medications_nodups.groupby(by="PATIENT")
data_medications_final = data_medications_grouped[["MED_BASE_COST", "MED_DISPENSES", "MED_TOTAL_COST", "MED_PAYER_COVERAGE"]].sum(min_count=1)
data_medications_final.rename(columns={"MED_BASE_COST": "OVERALL_MED_BASE_COST", 
                                         "MED_DISPENSES": "OVERALL_MED_DISPENSES", 
                                         "MED_TOTAL_COST": "OVERALL_MED_TOTAL_COST", 
                                         "MED_PAYER_COVERAGE": "OVERALL_MED_PAYER_COVERAGE"}, inplace=True)
data_medications_final["NUMBER_MEDICATIONS"] = data_medications_grouped[["MED_BASE_COST"]].count()
data_medications_final.shape

In [ ]:
data_medications_final

## Erstellung einer Patiententabelle (mit demografischen Daten und Diagnosedaten)

In [ ]:
# Merge Informationen zu Patienten aus data_costs und Diagnosen aus data_diagnoses über Patienten-ID
patient_information = pd.merge(data_costs[["PATIENT", "PAT_GENDER", "PAT_RACE"]], data_diagnoses, how="inner", left_on=["PATIENT"], right_on=["patient"])
# Doppelte Zeilen entfernen
patient_information.drop_duplicates(inplace=True)
# Doppelte Spalte mit Patienten-ID entfernen
patient_information.drop(columns=["patient"], inplace=True)
# Spalte diagnosis umbenennen
patient_information.rename(columns={"diagnosis": "DIAGNOSIS"}, inplace=True)
# Spalte mit Patienten-ID als Index
patient_information.set_index("PATIENT", inplace=True)

patient_information.shape

In [ ]:
patient_information

## Erstellung der Analysetabelle als Kombination der zuvor erstellten Hilfstabellen


In [ ]:
# Merge alle Tabellen in einzelnen Dataframe über Patienten-ID (Index)
patient_data = pd.merge(pd.merge(pd.merge(patient_information, 
                                          data_encounters_final, left_index=True, right_index=True), 
                                 data_procedures_final, left_index=True, right_index=True), 
                        data_medications_final, left_index=True, right_index=True)
patient_data.reset_index(inplace=True)
patient_data.shape

In [ ]:
patient_data

# EDA

## Übersicht über Patientenanzahl (rows) und beschreibende Attribute (columns)

In [ ]:
patient_data.shape

## Übersicht über die beschreibenden Attribute

In [ ]:
patient_data.columns

# Übersicht vorhandene Datentypen und Anzahl der Nullwerte

In [ ]:
patient_data.info()

## Erste Übersicht mit beschreibender Statistik (count, mean ...)

In [ ]:
patient_data.describe()

In [ ]:
patient_data.nunique(axis=0)

## Duplikate / doppelte Werte

In [ ]:
print("Anzahl doppelte Zeilen:", patient_data.duplicated(patient_data.columns).sum())

In [ ]:
# Entfernen von doppelten Werten (erster Wert bleibt bestehen)
patient_data.drop_duplicates(keep="first",inplace=True) 
print("Größe des Datensatzes nach dem Entfernen von doppelten Zeilen:", patient_data.shape)

## Vollständigkeit: Fehlende Werte (NULL / NaN)

In [ ]:
# Anteil von Nicht-NULL-Werten in jeder Spalte
patient_data.count()/len(patient_data) * 100

In [ ]:
# Anzahl NULL-Werte pro Spalte
patient_data.isnull().sum()

## Erstellen einer Kopie des Dataframes

In [ ]:
df = patient_data.copy()
df.head()

## Säulendiagramm "Gender"

In [ ]:
sns.countplot(x="PAT_GENDER", data=df, )

Ausgeglichene Anzahl von weiblichen und männlichen Patienten.

## Säulendiagramm "Diagnosen"

In [ ]:
sns.countplot(x="DIAGNOSIS", data=df)

Bei den meisten Patienten wurde keine Diagnose gestellt (none). Bei ca.150 Patienten wurde eine Diagnose gestellt, die aber nicht Brustkrebs war. Und nur bei sehr wenigen Patienten ergab sich die Diagnose Brustkrebs.

## Säulendiagramm "Ethnicity"

In [ ]:
sns.countplot(x="PAT_RACE", data=df)

Dieses Säulendiagramm zeigt, dass die meisten Patienten kaukasischer Abstammung waren. Andere Ethnien kamen nur in Ausnahmefällen vor. (Es müsste hier eine Normalisierung stattfinden, d.h. die gesamte Anzahl der Personen unterschiedlicher Ethnien müsste mit dem Patientenklientel verglichen werden.) 

## Anzahl der Arztbesuche pro Diagnose

In [ ]:
df.plot(kind='scatter', y = ('NUMBER_ENCOUNTERS'), x='DIAGNOSIS')

Es zeigt sich hier, dass Patienten mit Brustkrebs deutlich öfter einen Arzt aufsuchen als Patienten mit anderer oder keiner Diagnose.

## Ausgaben für Medikamente für die unterschiedlichen Diagnosen

In [ ]:
df.plot(kind='scatter', y = ('OVERALL_MED_TOTAL_COST'), x='DIAGNOSIS')

Die Ausgaben für Medikamente lassen keine Aussagen zu, um welche Diagnose es sich handelt.

## Berechnung der Kosten, die die Patienten mit den verschiedenen Diagnosen tragen müssen.

Hier eine Aufstellung wieviel die Medikamente insgesamt kosten und welcher Anteil von der Versicherung übernommen wird.

In [ ]:
df.groupby('DIAGNOSIS')['OVERALL_MED_TOTAL_COST', 'OVERALL_MED_PAYER_COVERAGE'].mean()

Hier eine Aufstellung wieviel die Arztbesuche insgesamt kosten und wieviel von den Versicherungen übernommen wird.

In [ ]:
df.groupby('DIAGNOSIS')['OVERALL_ENC_BASE_COST', 'OVERALL_ENC_PAYER_COVERAGE'].mean()

In [ ]:
groupby_obj = df.groupby('DIAGNOSIS')

In [ ]:
groupby_obj.mean()

In [ ]:
groupby_obj.mean().plot(kind='bar')

Berechnung wieviel der Kosten der Patient selber tragen muss (Medikamente und Arztbesuche)

In [ ]:
df.eval('PATIENT_COST = (OVERALL_ENC_BASE_COST - OVERALL_ENC_PAYER_COVERAGE) + (OVERALL_MED_TOTAL_COST - OVERALL_MED_PAYER_COVERAGE)', inplace = True)
df.columns

In [ ]:
groupby_obj = df.groupby('DIAGNOSIS')

In [ ]:
groupby_obj.mean()

In [ ]:
df_cost = df[['PATIENT_COST', 'DIAGNOSIS']]

In [ ]:
df_cost.head()

In [ ]:
groupby_obj = df_cost.groupby('DIAGNOSIS')

In [ ]:
groupby_obj.mean()

In [ ]:
groupby_obj.mean().plot(kind='bar')

In [ ]:
df_encounters = df[[ 'DIAGNOSIS', 'NUMBER_ENCOUNTERS']]

In [ ]:
df_encounters.head()

In [ ]:
groupby_obj = df_encounters.groupby('DIAGNOSIS')

In [ ]:
groupby_obj.mean()

In [ ]:
groupby_obj.mean().plot(kind='bar')

## Machine Learning

###Anhand der Kosten, die für den Patienten entstehen und der Anzahl der Arztbesuche soll auf die Diagnose der einzelnen Patienten geschlossen werden.

In [ ]:
df_ml = df[['DIAGNOSIS', 'NUMBER_ENCOUNTERS', 'PATIENT_COST']]

In [ ]:
df_ml.head()

In [ ]:
df_ml.groupby('DIAGNOSIS')

In [ ]:
df_ml.groupby('DIAGNOSIS').head()

In [ ]:
# entfernen von NaNs
df_rest = df_ml.dropna(axis=0)

In [ ]:
df_rest.head()

## Aufgrund der geringen Anzahl an vollständigen Datensätzen wird zunächst ein Decision Tree Modell versucht

In [ ]:
# Importing the required packages 
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

# Suppress warnings 
# (sometimes you might want to ignore warnings, that's how you can achieve this)
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Defining X and y
features = df_rest.columns.tolist()
features.remove('DIAGNOSIS')
X = df_rest[features]
y = df_rest.DIAGNOSIS

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=150, shuffle=True)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

Es stehen 68 vollständige Trainingsdatensätze und 30 vollständige Testdatensätze zur Verfügung.

Training eines Decision Trees mit dem gini-Index:

In [ ]:
# Creating the classifier object 
clf_gini = DecisionTreeClassifier(criterion = "gini", 
            max_depth=3, min_samples_leaf=5) 
# Performing training 
clf_gini.fit(X_train, y_train)

Vorhersage der Diagnose anhand der Testdaten:

In [ ]:
 # Predicton on test with giniIndex 
y_pred = clf_gini.predict(X_test) 
print("Predicted values:\n") 
print(y_pred)

Darstellung der Konfusionsmatrix und der Accuracy:

In [ ]:
print("-----"*15)
print("Confusion Matrix: \n", 
confusion_matrix(y_test, y_pred)) 

print("-----"*15)
print ("Accuracy : \n", 
accuracy_score(y_test,y_pred)*100) 
    
print("-----"*15)
print("Report : \n", 
classification_report(y_test, y_pred))

Graphische Darstellung der Konfusionsmatrix:

In [ ]:
# Evaluate the model with a confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, cmap='YlGnBu', annot=True, fmt='d', linewidths=.5);

## Ein weiterer Machine Learning Ansatz mit Hilfe einer Logistic Regression Analyse

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (11, 7)

Aufteilung in Trainings- und Testdaten sowie Training des Modells
Weiterhin graphische Darstellung der Konfusionsmatrix und der Accuracy.

In [ ]:
# Logistic Regression
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred_train = log_reg.predict(X_train)
y_pred = log_reg.predict(X_test)

# Print accuracy of our model
print("Accuracy on train set:", round(accuracy_score(y_train, y_pred_train), 2))
print("Accuracy on test set:", round(accuracy_score(y_test, y_pred), 2))
print("--------"*10)

# Print classification report of our model
print(classification_report(y_test, y_pred))
print("--------"*10)

# Evaluate the model with a confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, cmap='YlGnBu', annot=True, fmt='d', linewidths=.5);